In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import os
import numpy as np
import matplotlib.pyplot as plt
from shutil import copyfile
from scipy.io import loadmat



# Set up data paths

In [ ]:
images_path = "./JPEGImages"
labels_path = "./ImageSplits"
new_dataset_path = "./StanfordActionDataset"

# Create directories if they don't exist
if not(os.path.exists(new_dataset_path)):
    os.makedirs(new_dataset_path + '/' + 'train')
    os.makedirs(new_dataset_path + '/' + 'test')

# Move files to train and test directories
txts = os.listdir(labels_path)
for txt in txts:
    idx = txt[0:-4].rfind('_')
    class_name = txt[0:idx]
    if class_name in ['actions.tx', 'test.tx', 'train.tx']:
        continue
    train_or_test = txt[idx+1:-4]
    txt_contents = open(labels_path + '/' + txt).read()
    image_names  = txt_contents.split('\n')
    for image_name in image_names[0:-1]:
        if not(os.path.exists(new_dataset_path + '/' + train_or_test + '/' + class_name)):
            os.makedirs(new_dataset_path + '/' + train_or_test + '/' + class_name)
        copyfile(images_path + '/' + image_name,
                 new_dataset_path + '/' + train_or_test + '/' + class_name + '/' + image_name)

# Data transformations
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# Load data

In [ ]:
trainset = datasets.ImageFolder(root='./StanfordActionDataset/train/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = datasets.ImageFolder(root='./StanfordActionDataset/test/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)


## Set up model, optimizer, and loss function

In [ ]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(512, len(trainset.classes))  # Modify for the number of classes in Stanford 40 dataset
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()



##Training loop

In [ ]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader)}")

## test model

In [ ]:
# Test model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")

# Plot test batch images
def normalize_image(image):
    image_min = image.min()
    image_max = image.max()
    image.clamp_(min=image_min, max=image_max)
    image.add_(-image_min).div_(image_max - image_min + 1e-5)
    return image

def plot_images(images, labels, classes, normalize=True):
    n_images = len(images)
    rows = int(np.sqrt(n_images))
    cols = int(np.sqrt(n_images))

    fig = plt.figure(figsize=(20, 20))
    for i in range(rows*cols):
        ax = fig.add_subplot(rows, cols, i+1)
        image = images[i]
        if normalize:
            image = normalize_image(image)
        ax.imshow(image.permute(1, 2, 0).cpu().numpy())
        ax.set_title(classes[labels[i]])
        ax.axis('off')

classes = testset.classes
batch = next(iter(testloader))
plot_images(batch[0], batch[1], classes, normalize=True)
